<a href="https://colab.research.google.com/github/ElenaShargina/patterns/blob/master/%D0%9F%D0%BE%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5%20%D0%BF%D0%B0%D1%82%D1%82%D0%B5%D1%80%D0%BD%D1%8B/Chain_of_responsibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chain of responsibility / Цепочка обязанностей

Паттерн позволяет избежать привязки отправителя запроса к его получателю, давая шанс обработать запрос нескольким объектам. Связывает объекты-получатели в цепочку и передает запрос вдоль этой цепочки, пока его не обработают.

## Пример реализации

In [36]:
# Класс для построения дерева виджетов
class Component:
    def __init__(self,name):
        self.name = name
        self.children = []

    def add(self,instance):
        self.children = self.children + [instance,]

    def __str__(self):
        return f'Название: {self.name}, строка помощи: {self.text}\n'

    def display(self,tab=''):
        res = tab+str(self)
        if len(self.children)>0:
          for i in self.children:
              res += i.display(tab+'-')
        else:
          res = tab + str(self)
        return res

"""
Подмешиваемый в виджеты класс, содержащий текст подсказки и ссылку на родительский виджет - где будет
искать текст помощи в случае отсутствия текста помощи у себя
"""
class HelpHandler:
    def __init__(self, parent = None, help_text=None):
      self._next = parent
      self.text = help_text

    @property
    def has_text(self):
        if self.text!=None:
            return True
        else:
            return False

    def help(self):
        if self.has_text:
            print('HELP: ' + str(self.text))
        else:
            if self._next!=None:
                self._next.help()
            else:
                print('No help is found =(')

# Класс виджета
class Widget(Component, HelpHandler):
    def __init__(self,  name, parent=None, help_text=None):
        getattr(Component, '__init__')(self,name)
        getattr(HelpHandler, '__init__')(self,parent,help_text)
    
    def add(self,instance):
        instance._next = self
        getattr(Component, 'add')(self, instance)


# Построим дерево виджетов. У каких-то будет свой текст помощи, у каких-то нет. 
window = Widget('Main window', parent=None, help_text = 'Some help text for main window' )

column1 = Widget('Left column',  help_text = None)
button1 = Widget('Button Save',  help_text = 'Some help text for Save button')
column1.add(button1)
button2 = Widget('Button Discard',  help_text = 'Some help text for Discard button')
column1.add(button2)
window.add(column1)

column2 = Widget('Right column',  help_text = 'Some help text for Right column')
button3 = Widget('Button Load',  help_text = 'Some help text for Load button')
column2.add(button3)
button4 = Widget('Button New',  help_text = None)
column2.add(button4)
window.add(column2)

column3 = Widget('Footer',  help_text = None)
button5 = Widget('Button Exit',  help_text = None)
column3.add(button5)
window.add(column3)

print(window.display())

# Запросим текст помощи для кнопки Discard
# У нее есть свой собственный текст помощи
print('Текст помощи для кнопки Discard : ')
button2.help()
# Запросим текст помощи для кнопки New
# У нее нет своего текста, идем выше по дереву вплоть до Right column
print('Текст помощи для кнопки New : ')
button4.help()
# Запросим текст помощи для кнопки New
# У нее нет своего текста, идем выше по дереву вплоть до Main window
print('Текст помощи для кнопки Exit : ')
button5.help()

Название: Main window, строка помощи: Some help text for main window

Название: Main window, строка помощи: Some help text for main window
-Название: Left column, строка помощи: None
--Название: Button Save, строка помощи: Some help text for Save button
--Название: Button Discard, строка помощи: Some help text for Discard button
-Название: Right column, строка помощи: Some help text for Right column
--Название: Button Load, строка помощи: Some help text for Load button
--Название: Button New, строка помощи: None
-Название: Footer, строка помощи: None
--Название: Button Exit, строка помощи: None

Текст помощи для кнопки Discard : 
HELP: Some help text for Discard button
Текст помощи для кнопки New : 
HELP: Some help text for Right column
Текст помощи для кнопки Exit : 
HELP: Some help text for main window
